<a href="https://colab.research.google.com/github/Rakabi007/Bike-Scooter-Rebalancing-Optimization-For-Shared-memory/blob/main/bike_shared_rebalancing_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math, random, requests
import numpy as np, pandas as pd, geopandas as gpd
from shapely.geometry import Point
import networkx as nx
import folium
from folium import FeatureGroup, LayerControl


In [ ]:
# ----------------- Config  -----------------
SYSTEM      = "PORTLAND_BIKETOWN"   # or "NASHVILLE_BCYCLE", "NYC_CITIBIKE", "PORTLAND_BIKETOWN"
GBFS_ROOT   = None                  # <-- force auto-pick; don't reuse a prior value
TARGET_FILL = 0.45
STATION_CAP = 2500
K_NEAREST   = 3
SEED        = 42
BBOX = None

random.seed(SEED); np.random.seed(SEED)


# =========================
# GBFS loader (robust) + UTM projection
# =========================
SYSTEMS = {
    "NYC_CITIBIKE":      "https://gbfs.citibikenyc.com/gbfs/gbfs.json",         # docked
    "PORTLAND_BIKETOWN": "https://gbfs.biketownpdx.com/gbfs/gbfs.json",         # hybrid/dockless supports virtual stations
    "NASHVILLE_BCYCLE":  "https://gbfs.bcycle.com/bcycle_nashville/gbfs.json",  # may be sparse; still fine
}

def pick_gbfs_root(system, override=None):
    if override is not None and str(override).strip():
        return override
    if system in SYSTEMS:
        return SYSTEMS[system]
    raise ValueError("Unknown system and no GBFS_ROOT provided.")



In [ ]:
def get_feed_url(root_json, feed_name, lang="en"):
    data = root_json.get("data", {})
    feeds = data.get(lang, data[next(iter(data))])["feeds"]
    for f in feeds:
        if f["name"] == feed_name:
            return f["url"]
    raise KeyError(f"Feed '{feed_name}' not found in GBFS root")

def read_json(url):
    import requests
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.json()

def load_gbfs_stations(root_url):
    root = read_json(root_url)
    url_info   = get_feed_url(root, "station_information")
    url_status = get_feed_url(root, "station_status")

    info   = pd.DataFrame(read_json(url_info).get("data", {}).get("stations", []))
    status = pd.DataFrame(read_json(url_status).get("data", {}).get("stations", []))

    # Normalize IDs
    if "station_id" not in info.columns and "stationCode" in info.columns:
        info = info.rename(columns={"stationCode": "station_id"})
    if "station_id" not in status.columns and "stationCode" in status.columns:
        status = status.rename(columns={"stationCode": "station_id"})
    if "station_id" not in info.columns or "station_id" not in status.columns:
        raise KeyError("GBFS missing station_id/stationCode in one of the feeds")

    df = info.merge(status, on="station_id", how="inner", suffixes=("_info", "_status"))

    # Filter out virtual or non-installed / non-renting if present
    def to_bool(x):
        if isinstance(x, bool): return x
        if pd.isna(x): return True
        return str(x).strip().lower() in ("1","true","yes")
    if "is_virtual_station" in df.columns:
        df = df[~df["is_virtual_station"].apply(to_bool)]
    if "is_installed" in df.columns:
        df = df[df["is_installed"].apply(to_bool)]
    if "is_renting" in df.columns:
        df = df[df["is_renting"].apply(to_bool)]

    # Coerce lon/lat; some feeds use 'longitude'/'latitude'
    if not {"lon","lat"}.issubset(df.columns):
        swap = {}
        if "longitude" in df.columns: swap["longitude"] = "lon"
        if "latitude"  in df.columns: swap["latitude"]  = "lat"
        df = df.rename(columns=swap)
    for c in ("lon","lat"):
        if c not in df.columns:
            raise KeyError("No lon/lat in station_information")
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Bikes/docks/capacity (fallbacks); some dockless systems have no docks
    if "num_bikes_available" not in df.columns:
        df["num_bikes_available"] = 0
    if "num_docks_available" not in df.columns:
        df["num_docks_available"] = 0
    if "capacity" not in df.columns:
        df["capacity"] = df["num_bikes_available"].fillna(0) + df["num_docks_available"].fillna(0)

    # Clean types
    for c in ("num_bikes_available","num_docks_available","capacity"):
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    # Final tidy: drop invalid rows
    before = len(df)
    df = df.dropna(subset=["lon","lat"]).copy()
    # Require positive capacity (skip free-floating ‘virtual’ points)
    df = df[df["capacity"] > 0].copy()
    dropped = before - len(df)

    out = pd.DataFrame({
        "id":   df["station_id"].astype(str),
        "name": df.get("name", df["station_id"]),
        "lon":  df["lon"].astype(float),
        "lat":  df["lat"].astype(float),
        "cap":  df["capacity"].astype(int),
        "inv":  df["num_bikes_available"].astype(int),
        "docks": df["num_docks_available"].astype(int)
    })
    print(f"Loaded GBFS stations (after cleaning): {len(out)}  | dropped={dropped}")
    return out.reset_index(drop=True)


def safe_utm_crs_from_mean(lons, lats):
    """Fallback if estimate_utm_crs fails: compute UTM zone from mean lon/lat."""
    lon = float(np.nanmean(lons)); lat = float(np.nanmean(lats))
    zone = int((lon + 180) // 6) + 1
    epsg = 32600 + zone if lat >= 0 else 32700 + zone
    return f"EPSG:{epsg}"


In [ ]:
# # ---- use the loader ----
GBFS_ROOT = pick_gbfs_root(SYSTEM, GBFS_ROOT)
print(f"Using GBFS root for {SYSTEM}: {GBFS_ROOT}")  # sanity check
stations_ll = load_gbfs_stations(GBFS_ROOT)

In [ ]:

# Project to meters (UTM). Try GeoPandas auto; fallback to manual UTM if NaNs trigger an error.
gdf = gpd.GeoDataFrame(stations_ll, geometry=gpd.points_from_xy(stations_ll.lon, stations_ll.lat), crs=4326)
try:
    CRS_UTM = gdf.estimate_utm_crs()
except Exception:
    CRS_UTM = safe_utm_crs_from_mean(stations_ll.lon.values, stations_ll.lat.values)

gdf_m = gdf.to_crs(CRS_UTM)

stations = pd.DataFrame({
    "id": stations_ll["id"],
    "name": stations_ll["name"],
    "lon": stations_ll["lon"],
    "lat": stations_ll["lat"],
    "xm":  gdf_m.geometry.x.values,
    "ym":  gdf_m.geometry.y.values,
    "cap": stations_ll["cap"],
    "inv": stations_ll["inv"],
})


In [ ]:
# Build targets and surplus/deficit as before
stations["target"]  = (TARGET_FILL * stations["cap"]).round().astype(int)
stations["surplus"] = (stations["inv"] - stations["target"]).clip(lower=0).astype(int)
stations["deficit"] = (stations["target"] - stations["inv"]).clip(lower=0).astype(int)
print(f"Loaded stations: {len(stations)} (system={SYSTEM})")



In [ ]:
# ----------------- Simple K-nearest greedy rebalancing -----------------
donors    = stations[stations.surplus > 0][["id","xm","ym","surplus"]].copy()
receivers = stations[stations.deficit > 0][["id","xm","ym","deficit"]].copy()

def euc(a,b): return math.hypot(a[0]-b[0], a[1]-b[1])

shipments = []  # (from_id, to_id, qty)

if not donors.empty and not receivers.empty:
    # build K nearest receiver lists for each donor
    recv_xy = receivers.set_index("id")[["xm","ym","deficit"]].to_dict("index")
    donor_rows = donors.to_dict("records")

    # maintain mutable deficits
    rem_def = {rid: r["deficit"] for rid, r in recv_xy.items()}

    for d in donor_rows:
        d_id, dx, dy, s = d["id"], d["xm"], d["ym"], int(d["surplus"])
        if s <= 0: continue
        # nearest K receivers by meters
        dists = sorted(((rid, euc((dx,dy),(recv_xy[rid]["xm"], recv_xy[rid]["ym"])))
                        for rid in recv_xy.keys()), key=lambda t: t[1])[:max(1, K_NEAREST)]
        for rid, _ in dists:
            if s <= 0: break
            need = rem_def.get(rid, 0)
            if need <= 0: continue
            q = min(s, need)
            shipments.append((d_id, rid, int(q)))
            s       -= q
            rem_def[rid] = need - q


In [ ]:
# Plan totals for popups
plan_pick = {}
plan_drop = {}
for sid, tid, q in shipments:
    plan_pick[sid] = plan_pick.get(sid,0) + q
    plan_drop[tid] = plan_drop.get(tid,0) + q

stations["to_pick"] = stations["id"].map(plan_pick).fillna(0).astype(int)
stations["to_drop"] = stations["id"].map(plan_drop).fillna(0).astype(int)



In [ ]:
# ----------------- KPIs / validation -----------------
ship_total = sum(q for _,_,q in shipments)
print(f"\nFlow check: shipped={ship_total}, surplus={int(stations.surplus.sum())}, deficit={int(stations.deficit.sum())}")

tmp = stations.set_index("id")[["inv","cap"]].copy()
for sid, tid, q in shipments:
    tmp.at[sid, "inv"] -= q
    tmp.at[tid, "inv"] += q
viol_low  = int((tmp["inv"] < 0).sum())
viol_high = int((tmp["inv"] > tmp["cap"]).sum())
mad_after = (tmp["inv"] - (TARGET_FILL*tmp["cap"]).round()).abs().mean()
print(f"Post-ship bounds violations → below 0: {viol_low}, above cap: {viol_high}")
print(f"Mean |inv-target| after shipping: {mad_after:.2f} bikes")

In [ ]:
# shipment stats
if shipments:
    S = pd.DataFrame(shipments, columns=["from_id","to_id","qty"])
    idx = stations.set_index("id")
    S["sx"], S["sy"] = idx.loc[S["from_id"], "xm"].values, idx.loc[S["from_id"], "ym"].values
    S["tx"], S["ty"] = idx.loc[S["to_id"],   "xm"].values, idx.loc[S["to_id"],   "ym"].values
    S["dist_km"] = np.hypot(S.sx - S.tx, S.sy - S.ty) / 1000.0
    print("\nShipment distance summary (km) and qty:")
    print(S[["qty","dist_km"]].describe().to_string(index=False))
    S[["from_id","to_id","qty","dist_km"]].to_csv("shipments.csv", index=False)
    print("Wrote shipments.csv")


In [ ]:
# ----------------- Map (two layers) -----------------
# center
cy, cx = stations["lat"].median(), stations["lon"].median()
m = folium.Map(location=[cy, cx], zoom_start=12, tiles="cartodbpositron")
fg_st = FeatureGroup(name="Stations", show=True)
fg_sh = FeatureGroup(name="Shipments (greedy K-nearest)", show=True)

for _, r in stations.iterrows():
    color = "#1a9850" if r.surplus>0 else "#d73027" if r.deficit>0 else "#666"
    popup = (f"<b>{r.name}</b> (id={r.id})<br>"
             f"cap={r.cap}, inv={r.inv}, target={int(TARGET_FILL*r.cap)}<br>"
             f"surplus={r.surplus}, deficit={r.deficit}<br>"
             f"plan pick={r.to_pick}, drop={r.to_drop}")
    folium.CircleMarker([r.lat, r.lon], radius=3.5, color=color, fill=True, fill_opacity=0.95,
                        popup=popup).add_to(fg_st)


In [ ]:
# draw shipments (thin blue)
for sid, tid, q in shipments:
    a = stations.loc[stations.id==sid, ["lat","lon"]].iloc[0].tolist()
    b = stations.loc[stations.id==tid, ["lat","lon"]].iloc[0].tolist()
    folium.PolyLine([a,b], color="#2b8cbe", weight=2, opacity=0.5,
                    tooltip=f"{sid} → {tid} (q={q})").add_to(fg_sh)

fg_st.add_to(m); fg_sh.add_to(m)
LayerControl(collapsed=False).add_to(m)
out = f"rebalancing_lite_{SYSTEM.lower()}.html"
m.save(out)
print("Map saved:", out)



In [ ]:
# ----------------- Notes -----------------
# If you later want VRP truck routes, plug the 'shipments' list into your OR-Tools section.






Using GBFS root for PORTLAND_BIKETOWN: https://gbfs.biketownpdx.com/gbfs/gbfs.json
Loaded GBFS stations (after cleaning): 254  | dropped=0
Loaded stations: 254 (system=PORTLAND_BIKETOWN)

Flow check: shipped=107, surplus=145, deficit=599
Post-ship bounds violations → below 0: 0, above cap: 1
Mean |inv-target| after shipping: 2.09 bikes

Shipment distance summary (km) and qty:
      qty   dist_km
60.000000 60.000000
 1.783333  0.612019
 1.026623  0.312627
 1.000000  0.102715
 1.000000  0.406652
 2.000000  0.539201
 2.000000  0.793715
 6.000000  1.458119
Wrote shipments.csv
Map saved: rebalancing_lite_portland_biketown.html
